In [0]:
from pyspark.sql.functions import col, asc, regexp_replace, collect_set,size,array,array_except,lit,array_union,array_distinct,array_remove, concat_ws
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DoubleType,IntegerType, BooleanType
from pyspark import StorageLevel
import json
from pyspark.ml.fpm import FPGrowth


In [0]:
outpatient = spark.read.parquet("abfss://cms@datasciencecms.dfs.core.windows.net/sample_2016_2022_repartition/outpatient_by_provider")


In [0]:
inpatient = spark.read.parquet("abfss://cms@datasciencecms.dfs.core.windows.net/sample_2016_2022_repartition/inpatient_by_provider")


In [0]:
outpatient.createOrReplaceTempView("outpatient")
inpatient.createOrReplaceTempView("inpatient")


In [0]:
outpatient.columns

['desy_sort_key_claim',
 'claim_no_claim',
 'PRVDR_NUM',
 'clm_thru_dt_claim',
 'NCH_NEAR_LINE_REC_IDENT_CD',
 'nch_clm_type_cd_claim',
 'CLAIM_QUERY_CODE',
 'CLM_FAC_TYPE_CD',
 'CLM_SRVC_CLSFCTN_TYPE_CD',
 'CLM_FREQ_CD',
 'FI_NUM',
 'CLM_MDCR_NON_PMT_RSN_CD',
 'CLM_PMT_AMT',
 'NCH_PRMRY_PYR_CLM_PD_AMT',
 'NCH_PRMRY_PYR_CD',
 'PRVDR_STATE_CD',
 'ORG_NPI_NUM',
 'SRVC_LOC_NPI_NUM',
 'AT_PHYSN_UPIN',
 'AT_PHYSN_NPI',
 'AT_PHYSN_SPCLTY_CD',
 'OP_PHYSN_UPIN',
 'OP_PHYSN_NPI',
 'OP_PHYSN_SPCLTY_CD',
 'OT_PHYSN_UPIN',
 'OT_PHYSN_NPI',
 'OT_PHYSN_SPCLTY_CD',
 'RNDRNG_PHYSN_NPI',
 'RNDRNG_PHYSN_SPCLTY_CD',
 'RFR_PHYSN_NPI',
 'RFR_PHYSN_SPCLTY_CD',
 'CLM_MCO_PD_SW',
 'PTNT_DSCHRG_STUS_CD',
 'CLM_TOT_CHRG_AMT',
 'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM',
 'NCH_PROFNL_CMPNT_CHRG_AMT',
 'PRNCPAL_DGNS_CD',
 'ICD_DGNS_CD1',
 'ICD_DGNS_CD2',
 'ICD_DGNS_CD3',
 'ICD_DGNS_CD4',
 'ICD_DGNS_CD5',
 'ICD_DGNS_CD6',
 'ICD_DGNS_CD7',
 'ICD_DGNS_CD8',
 'ICD_DGNS_CD9',
 'ICD_DGNS_CD10',
 'ICD_DGNS_CD11',
 'ICD_DGNS_CD12'

In [0]:
inpatient.columns

['desy_sort_key_claim',
 'claim_no_claim',
 'PRVDR_NUM',
 'clm_thru_dt_claim',
 'NCH_NEAR_LINE_REC_IDENT_CD',
 'nch_clm_type_cd_claim',
 'CLAIM_QUERY_CODE',
 'CLM_FAC_TYPE_CD',
 'CLM_SRVC_CLSFCTN_TYPE_CD',
 'CLM_FREQ_CD',
 'FI_NUM',
 'CLM_MDCR_NON_PMT_RSN_CD',
 'CLM_PMT_AMT',
 'NCH_PRMRY_PYR_CLM_PD_AMT',
 'NCH_PRMRY_PYR_CD',
 'FI_CLM_ACTN_CD',
 'PRVDR_STATE_CD',
 'ORG_NPI_NUM',
 'AT_PHYSN_UPIN',
 'AT_PHYSN_NPI',
 'AT_PHYSN_SPCLTY_CD',
 'OP_PHYSN_UPIN',
 'OP_PHYSN_NPI',
 'OP_PHYSN_SPCLTY_CD',
 'OT_PHYSN_UPIN',
 'OT_PHYSN_NPI',
 'OT_PHYSN_SPCLTY_CD',
 'RNDRNG_PHYSN_NPI',
 'RNDRNG_PHYSN_SPCLTY_CD',
 'CLM_MCO_PD_SW',
 'PTNT_DSCHRG_STUS_CD',
 'CLM_PPS_IND_CD',
 'CLM_TOT_CHRG_AMT',
 'CLM_ADMSN_DT',
 'CLM_IP_ADMSN_TYPE_CD',
 'CLM_SRC_IP_ADMSN_CD',
 'NCH_PTNT_STATUS_IND_CD',
 'CLM_PASS_THRU_PER_DIEM_AMT',
 'NCH_BENE_IP_DDCTBL_AMT',
 'NCH_BENE_PTA_COINSRNC_LBLTY_AM',
 'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM',
 'NCH_PROFNL_CMPNT_CHRG_AMT',
 'NCH_IP_NCVRD_CHRG_AMT',
 'CLM_TOT_PPS_CPTL_AMT',
 'CLM_PPS_CPT

In [0]:
final_inpatient = spark.sql("select desy_sort_key, claim_no_claim,clm_thru_dt_claim,PRVDR_NUM,PRVDR_STATE_CD, explode(array_compact(array(PRNCPAL_DGNS_CD,ICD_DGNS_CD1,\
 ICD_DGNS_CD2,\
 ICD_DGNS_CD3,\
 ICD_DGNS_CD4,\
 ICD_DGNS_CD5,\
 ICD_DGNS_CD6,\
 ICD_DGNS_CD7,\
 ICD_DGNS_CD8,\
 ICD_DGNS_CD9,\
 ICD_DGNS_CD10,\
 ICD_DGNS_CD11,\
 ICD_DGNS_CD12,\
 ICD_DGNS_CD13,\
 ICD_DGNS_CD14,\
 ICD_DGNS_CD15,\
 ICD_DGNS_CD16,\
 ICD_DGNS_CD17,\
 ICD_DGNS_CD18,\
 ICD_DGNS_CD19,\
 ICD_DGNS_CD20,\
 ICD_DGNS_CD21,\
 ICD_DGNS_CD22,\
 ICD_DGNS_CD23,\
 ICD_DGNS_CD24,\
 ICD_DGNS_CD25))) as DIAGNOSIS from inpatient where PRVDR_STATE_CD='46'")

In [0]:
final_outpatient = spark.sql("select desy_sort_key,claim_no_claim,clm_thru_dt_claim,PRVDR_NUM,PRVDR_STATE_CD, explode(array_compact(array(PRNCPAL_DGNS_CD,ICD_DGNS_CD1,\
 ICD_DGNS_CD2,\
 ICD_DGNS_CD3,\
 ICD_DGNS_CD4,\
 ICD_DGNS_CD5,\
 ICD_DGNS_CD6,\
 ICD_DGNS_CD7,\
 ICD_DGNS_CD8,\
 ICD_DGNS_CD9,\
 ICD_DGNS_CD10,\
 ICD_DGNS_CD11,\
 ICD_DGNS_CD12,\
 ICD_DGNS_CD13,\
 ICD_DGNS_CD14,\
 ICD_DGNS_CD15,\
 ICD_DGNS_CD16,\
 ICD_DGNS_CD17,\
 ICD_DGNS_CD18,\
 ICD_DGNS_CD19,\
 ICD_DGNS_CD20,\
 ICD_DGNS_CD21,\
 ICD_DGNS_CD22,\
 ICD_DGNS_CD23,\
 ICD_DGNS_CD24,\
 ICD_DGNS_CD25))) as DIAGNOSIS from outpatient where PRVDR_STATE_CD='46'")

In [0]:
final_total = final_outpatient.unionAll(final_inpatient)

In [0]:
final_total.createOrReplaceTempView("final_total")


In [0]:
#Asthma                   ICD-10: J45
#Acute Bronchitis.    ICD-10: J20
#COPD                    ICD-10: J44
#Dyspnea                ICD-10: R06
#Emesis.                 ICD-10: R11
#Coccidioidomycosis ICD-10: B38

In [0]:
#r asthma (ICD-10: J45), acute bronchitis (ICD-10: J20), COPD (ICD-10: J44), and dyspnea (ICD-10: R06)

In [0]:
def make_map(input):
    if input.startswith("J45"):
        return "Asmtha"
    if input.startswith("J20"):
        return "COPD"
    if input.startswith("R06"):
        return "Dyspnea"
    if input.startswith("R11"):
        return "Emesis"
    if input.startswith("B38"):
        return "Coccidioidomycosis"

In [0]:
def make_map_group(input):
    if input.startswith("J45"):
        return "A_AB_COPD_DY"
    if input.startswith("J20"):
        return "A_AB_COPD_DY"
    if input.startswith("J44"):
        return "A_AB_COPD_DY"
    if input.startswith("R06"):
        return "A_AB_COPD_DY"
    if input.startswith("R11"):
        return "Emesis"
    if input.startswith("B38"):
        return "Coccidioidomycosis"

In [0]:
map_udf = udf(lambda x: make_map(x),StringType())
map_group_udf = udf(lambda x: make_map_group(x),StringType())



In [0]:
final_total.columns

['desy_sort_key',
 'claim_no_claim',
 'clm_thru_dt_claim',
 'PRVDR_NUM',
 'PRVDR_STATE_CD',
 'DIAGNOSIS']

In [0]:
map_diagnosis_final = final_total.withColumn("mapped_diagnosis",map_udf("DIAGNOSIS")).withColumn("related_diagnosis",map_group_udf("DIAGNOSIS"))

In [0]:
map_diagnosis_final.createOrReplaceTempView("map_diagnosis_final")


In [0]:
counts_day_detail = spark.sql("select count( distinct desy_sort_key) as patient_count,mapped_diagnosis, year(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_year, month(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_month, day(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_day,PRVDR_NUM,PRVDR_STATE_CD from  map_diagnosis_final where mapped_diagnosis is not null group by mapped_diagnosis,clm_thru_dt_claim, PRVDR_NUM,PRVDR_STATE_CD having patient_count>10 order by clm_thru_dt_claim asc  ")

In [0]:
counts_day_detail.first()

Row(patient_count=12, mapped_diagnosis='Emesis', claim_year=2017, claim_month=1, claim_day=9, PRVDR_NUM='460009', PRVDR_STATE_CD='46')

In [0]:
counts_day_detail.count()

1200

In [0]:
#counts_day_detail.repartition(1).write.option("header", "true").csv("abfss://cms@datasciencecms.dfs.core.windows.net/capstone/counts_day_detail")


In [0]:
counts_day_aggregate = spark.sql("select count( distinct desy_sort_key) as patient_count,related_diagnosis, year(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_year, month(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_month, day(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_day,PRVDR_NUM,PRVDR_STATE_CD from  map_diagnosis_final where related_diagnosis is not null group by related_diagnosis,clm_thru_dt_claim, PRVDR_NUM,PRVDR_STATE_CD having patient_count>10 order by clm_thru_dt_claim asc  ")

In [0]:
#counts_day_aggregate.repartition(1).write.option("header", "true").csv("abfss://cms@datasciencecms.dfs.core.windows.net/capstone/counts_day_aggregate")


In [0]:
counts_week_detail = spark.sql("select count( distinct desy_sort_key) as patient_count,mapped_diagnosis,  extract(week FROM to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_week,PRVDR_NUM,PRVDR_STATE_CD from  map_diagnosis_final where mapped_diagnosis is not null group by mapped_diagnosis,claim_week, PRVDR_NUM,PRVDR_STATE_CD having patient_count>10 order by claim_week asc  ")

In [0]:
counts_week_detail.first()

Row(patient_count=17, mapped_diagnosis='Asmtha', claim_week=1, PRVDR_NUM='460013', PRVDR_STATE_CD='46')

In [0]:
counts_week_detail.count()

2207

In [0]:
#counts_week_detail.repartition(1).write.option("header", "true").csv("abfss://cms@datasciencecms.dfs.core.windows.net/capstone/counts_week_detail")


In [0]:
counts_week_aggregate= spark.sql("select count( distinct desy_sort_key) as patient_count,related_diagnosis,  extract(week FROM to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_week,PRVDR_NUM,PRVDR_STATE_CD from  map_diagnosis_final where mapped_diagnosis is not null group by related_diagnosis,claim_week, PRVDR_NUM,PRVDR_STATE_CD having patient_count>10 order by claim_week asc  ")

In [0]:
counts_week_aggregate.first()

Row(patient_count=12, related_diagnosis='A_AB_COPD_DY', claim_week=1, PRVDR_NUM='461819', PRVDR_STATE_CD='46')

In [0]:
counts_week_aggregate.count()

1903

In [0]:
counts_week_aggregate.repartition(1).write.option("header", "true").csv("abfss://cms@datasciencecms.dfs.core.windows.net/capstone/counts_group_week_aggregate")


In [0]:
counts_month_detiail = spark.sql("select count( distinct desy_sort_key) as patient_count,mapped_diagnosis,  month(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_month,PRVDR_NUM,PRVDR_STATE_CD from  map_diagnosis_final where mapped_diagnosis is not null group by mapped_diagnosis,claim_month, PRVDR_NUM,PRVDR_STATE_CD having patient_count>10 order by claim_month asc  ")

  File <command-369920238023872>, line 6
    if input.startswith("J44")
                              ^
SyntaxError: expected ':'


In [0]:
counts_month_detiail.first()

Row(patient_count=33, mapped_diagnosis='Dyspnea', claim_month=1, PRVDR_NUM='460058', PRVDR_STATE_CD='46')

In [0]:
counts_month_detiail.count()

871

In [0]:
#counts_month_detiail.repartition(1).write.option("header", "true").csv("abfss://cms@datasciencecms.dfs.core.windows.net/capstone/counts_inpatient_month_detiail")


In [0]:
counts_month_aggregate = spark.sql("select count( distinct desy_sort_key) as patient_count,related_diagnosis,  month(to_date(clm_thru_dt_claim,'yyyyMMdd')) as claim_month,PRVDR_NUM,PRVDR_STATE_CD from  map_diagnosis_final where mapped_diagnosis is not null group by related_diagnosis,claim_month, PRVDR_NUM,PRVDR_STATE_CD having patient_count>10 order by claim_month asc  ")

In [0]:
counts_month_aggregate.first()

Row(patient_count=14, related_diagnosis='AirQualityCondition', claim_month=1, PRVDR_NUM='460054', PRVDR_STATE_CD='46')

In [0]:
counts_month_aggregate.count()

560

In [0]:
#counts_month_aggregate.repartition(1).write.option("header", "true").csv("abfss://cms@datasciencecms.dfs.core.windows.net/capstone/counts_inpatient_month_aggregate")
